In [1]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
import model_engine
import boto3
import  numpy as np
from functions_for_onboarding import *

In [9]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, Iterable, List, Set, Union

import numpy as np
import pandas as pd

from feature_engine_parts.fe_parts_V2.mappers import converters

# --------------------------------------------------------------------------------------
# Converter registry + helpers
# --------------------------------------------------------------------------------------

CONVERTER_REGISTRY = {
    "MappingBase": converters.MappingBase,
    "DateConverterV2": converters.DateConverterV2,
    "NumericConverterV2": converters.NumericConverterV2,
    "StringConverterV2": converters.StringConverterV2,
}


def _build_converter(spec: Dict[str, Any]):
    if not isinstance(spec, dict):
        raise TypeError(f"spec must be a dict, got {type(spec)}")

    type_name = spec.get("type")
    params = spec.get("params", {}) or {}

    if type_name not in CONVERTER_REGISTRY:
        known = ", ".join(sorted(CONVERTER_REGISTRY.keys()))
        raise ValueError(f"Unknown converter type '{type_name}'. Known types: {known}")

    if not isinstance(params, dict):
        raise TypeError(f"spec['params'] must be a dict, got {type(params)}")

    cls = CONVERTER_REGISTRY[type_name]
    return type_name, cls(**params)


def apply_converter_spec(df: pd.DataFrame, spec: Dict[str, Any], *, verbose: bool = True) -> pd.DataFrame:
    type_name, converter = _build_converter(spec)

    if verbose:
        raw_feature = getattr(converter, "raw_feature", None)
        new_feature = getattr(converter, "new_feature", None)
        print(f"[apply_converter_spec] Using {type_name} | raw_feature={raw_feature!r} -> new_feature={new_feature!r}")

    return converter.transform(df)


def _output_feature_name_from_spec(spec: Dict[str, Any]) -> str | None:
    params = spec.get("params", {}) or {}
    return params.get("new_feature") or params.get("raw_feature")


def build_output_feature_allowlist(specs: List[Dict[str, Any]]) -> Set[str]:
    out: Set[str] = set()
    for spec in specs:
        name = _output_feature_name_from_spec(spec)
        if name:
            out.add(name)
    return out


# --------------------------------------------------------------------------------------
# 1) Synthetic DATE_OF_REQUEST generator based on DATE_REPORTED grouped by ZEST_KEY
# --------------------------------------------------------------------------------------

def create_synthetic_date_of_request_simple(
    df: pd.DataFrame,
    *,
    zest_key_col: str = "ZEST_KEY",
    date_reported_col: str = "DATE_REPORTED",
    zest_key_out: str = "ZEST_KEY",
    rpt_col: str = "rptDate",
    out_col: str = "date_of_request",
    date_reported_format: str | None = "%m%d%Y",
    seed: int | None = None,
    verbose: bool = False,
) -> pd.DataFrame:
    out = df.copy()
    ## apply zest ke and date fix 
    out = apply_converter_spec(
        out,
        {"type": "StringConverterV2", "params": {"raw_feature": zest_key_col, "new_feature": zest_key_out}},
        verbose=verbose,
    )

    date_spec: Dict[str, Any] = {
        "type": "DateConverterV2",
        "params": {"raw_feature": date_reported_col, "new_feature": rpt_col},
    }
    if date_reported_format:
        date_spec["params"]["format"] = date_reported_format
    out = apply_converter_spec(out, date_spec, verbose=verbose)

    rng = np.random.default_rng(seed)
    # max rpt date
    max_rpt = out.groupby(zest_key_out, dropna=False)[rpt_col].max()

    ## mapping to random date for request
    offsets = pd.Series(rng.integers(1, 4, size=len(max_rpt)), index=max_rpt.index)

    dor_map = (max_rpt + offsets.map(lambda m: pd.DateOffset(months=int(m)))).rename(out_col)
    out[out_col] = out[zest_key_out].map(dor_map)

    return out
def load_mapping_specs_from_json(path: Union[str, Path], *, key: str = "mapping") -> List[Dict[str, Any]]:
    p = Path(path)
    with p.open("r", encoding="utf-8") as f:
        obj = json.load(f)

    if key not in obj:
        raise KeyError(f"JSON missing key '{key}'. Keys found: {list(obj.keys())}")

    specs = obj[key]
    if not isinstance(specs, list):
        raise TypeError(f"json['{key}'] must be a list, got {type(specs)}")

    return specs


# --------------------------------------------------------------------------------------
# 3) Apply mappings from JSON + filter outputs
#    (includes DATE_REPORTED as a feature to keep, per your request)
# --------------------------------------------------------------------------------------

def apply_mapping_json_and_filter(
    df: pd.DataFrame,
    mapping_json_path: Union[str, Path],
    *,
    exclude_raw_features: Iterable[str] = (),
    exclude_new_features: Iterable[str] = (),
    extra_keep: Iterable[str] = (),
    mapping_key: str = "mapping",
    always_keep: Iterable[str] = ("DATE_REPORTED",),
    strict: bool = False,
    verbose: bool = True,
) -> pd.DataFrame:
    specs = load_mapping_specs_from_json(mapping_json_path, key=mapping_key)

    exclude_raw = set(exclude_raw_features)
    exclude_new = set(exclude_new_features)

    def _spec_included(spec: Dict[str, Any]) -> bool:
        params = spec.get("params", {}) or {}
        raw = params.get("raw_feature")
        new = params.get("new_feature")
        if raw and raw in exclude_raw:
            return False
        if new and new in exclude_new:
            return False
        return True

    specs_to_apply = [s for s in specs if _spec_included(s)]

    out = df.copy()
    for i, spec in enumerate(specs_to_apply, start=1):
        if verbose:
            print(f"\n[apply_mapping_json_and_filter] Step {i}/{len(specs_to_apply)}")
        out = apply_converter_spec(out, spec, verbose=verbose)

    keep = build_output_feature_allowlist(specs_to_apply) | set(always_keep) | set(extra_keep)

    missing = [c for c in keep if c not in out.columns]
    if missing and strict:
        raise KeyError(f"Missing expected columns after mapping: {missing}")

    keep_in_df = [c for c in keep if c in out.columns]
    return out.loc[:, keep_in_df].copy()

from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Set, Union

import pandas as pd

# -----------------------------------------------------------------------------
# 1) Preprocessor registry
#    - Add/remove types based on what appears in your JSON preprocess section.
# -----------------------------------------------------------------------------
from feature_engine_parts.fe_parts_V2.preprocessors.date_diff import DateDiffV2
from feature_engine_parts.fe_parts_V2.preprocessors.coalesce import CoalesceV2
from feature_engine_parts.fe_parts_V2.preprocessors.bivariate_compute import BivariateComputeV2
from feature_engine_parts.fe_parts_V2.preprocessors.dynamic_placeholder import DynamicPlaceholderV2
from feature_engine_parts.fe_parts_V2.preprocessors.filter import FilterV2
from feature_engine_parts.fe_parts_V2.preprocessors.payment_pattern_aggregator import PaymentPatternsAggregatorV2
from feature_engine_parts.fe_parts_V2.preprocessors.row_dropper import RowDropperV2
def build_preprocessor_registry():
    return {
        "DateDiffV2": DateDiffV2,
        "CoalesceV2": CoalesceV2,
        "BivariateComputeV2": BivariateComputeV2,
        "DynamicPlaceholderV2": DynamicPlaceholderV2,
        "FilterV2": FilterV2,
        "PaymentPatternsAggregatorV2": PaymentPatternsAggregatorV2,
        "RowDropperV2": RowDropperV2,  # <-- add this
    }


# -----------------------------------------------------------------------------
# 2) JSON loading helpers
# -----------------------------------------------------------------------------
def load_specs_from_json(path: Union[str, Path], *, key: str) -> List[Dict[str, Any]]:
    p = Path(path)
    with p.open("r", encoding="utf-8") as f:
        obj = json.load(f)

    if key not in obj:
        raise KeyError(f"JSON missing key '{key}'. Keys found: {list(obj.keys())}")

    specs = obj[key]
    if not isinstance(specs, list):
        raise TypeError(f"json['{key}'] must be a list, got {type(specs)}")

    return specs


def _output_feature_name_from_spec(spec: Dict[str, Any]) -> Optional[str]:
    params = spec.get("params", {}) or {}
    # preprocessors often write to "new_feature"
    return params.get("new_feature") or params.get("raw_feature")


def build_output_feature_allowlist(specs: List[Dict[str, Any]]) -> Set[str]:
    out: Set[str] = set()
    for spec in specs:
        name = _output_feature_name_from_spec(spec)
        if name:
            out.add(name)
    return out


# -----------------------------------------------------------------------------
# 3) Build + apply a single preprocessor spec
# -----------------------------------------------------------------------------
def _build_preprocessor(spec: Dict[str, Any], registry: Dict[str, Any]):
    if not isinstance(spec, dict):
        raise TypeError(f"spec must be a dict, got {type(spec)}")

    type_name = spec.get("type")
    params = spec.get("params", {}) or {}

    if type_name not in registry:
        known = ", ".join(sorted(registry.keys()))
        raise ValueError(f"Unknown preprocessor type '{type_name}'. Known types: {known}")

    if not isinstance(params, dict):
        raise TypeError(f"spec['params'] must be a dict, got {type(params)}")

    cls = registry[type_name]
    return type_name, cls(**params)


def apply_preprocess_spec(df: pd.DataFrame, spec: Dict[str, Any], *, registry: Dict[str, Any], verbose: bool = True) -> pd.DataFrame:
    type_name, obj = _build_preprocessor(spec, registry)

    if verbose:
        params = spec.get("params", {}) or {}
        print(f"[apply_preprocess_spec] {type_name} params={list(params.keys())}")

    return obj.transform(df)


# -----------------------------------------------------------------------------
# 4) Apply full preprocess section (the main function you want)
# -----------------------------------------------------------------------------
def apply_preprocess_json_and_filter(
    mapped_df: pd.DataFrame,
    mapping_json_path: Union[str, Path],
    *,
    preprocess_key: str = "preprocess",
    # Keep behavior (optional): if you want to only return "mapped cols + new preprocess outputs"
    keep_original_columns: bool = True,
    extra_keep: Iterable[str] = (),
    strict_keep: bool = False,
    verbose: bool = True,
) -> pd.DataFrame:
    """
    Apply FE2 preprocess steps IN ORDER to an already-mapped df.

    keep_original_columns=True:
        returns (original mapped_df columns) + (preprocess-created outputs) + extra_keep

    keep_original_columns=False:
        returns only preprocess outputs + extra_keep
    """
    registry = build_preprocessor_registry()
    specs = load_specs_from_json(mapping_json_path, key=preprocess_key)

    out = mapped_df.copy()
    original_cols = list(out.columns)

    for i, spec in enumerate(specs, start=1):
        if verbose:
            print(f"\n[apply_preprocess_json_and_filter] Step {i}/{len(specs)} | type={spec.get('type')}")
        out = apply_preprocess_spec(out, spec, registry=registry, verbose=verbose)

    preprocess_outputs = build_output_feature_allowlist(specs)

    if keep_original_columns:
        keep = set(original_cols) | preprocess_outputs | set(extra_keep)
    else:
        keep = preprocess_outputs | set(extra_keep)

    missing = [c for c in keep if c not in out.columns]
    if missing and strict_keep:
        raise KeyError(f"Missing expected columns after preprocess: {missing}")

    keep_in_df = [c for c in keep if c in out.columns]
    return out.loc[:, keep_in_df].copy()


# -----------------------------------------------------------------------------
# 5) Convenience wrapper: mapping then preprocess (optional)
# -----------------------------------------------------------------------------
def apply_mapping_then_preprocess(
    df_raw: pd.DataFrame,
    mapping_json_path: Union[str, Path],
    *,
    mapping_func,  # pass your existing apply_mapping_json_and_filter
    mapping_kwargs: Optional[Dict[str, Any]] = None,
    preprocess_kwargs: Optional[Dict[str, Any]] = None,
) -> pd.DataFrame:
    mapping_kwargs = mapping_kwargs or {}
    preprocess_kwargs = preprocess_kwargs or {}

    mapped = mapping_func(df=df_raw, mapping_json_path=mapping_json_path, **mapping_kwargs)
    preprocessed = apply_preprocess_json_and_filter(mapped, mapping_json_path, **preprocess_kwargs)
    return preprocessed


# Create S3 Handler Object and Load Parsed and Processed for inquiry trade collection bankruptcy

In [5]:
from zestio import handler
import pandas as pd
bucket_name="power-client-data-staging"
s3 = handler.S3Handler(bucket_name=bucket_name)
archive_date = 'ARCHIVE_DATE=2025-03-31'
def grab_parsed_data(s3, archive_date = archive_date, PULL_NAME='PULL_NAME=20260118_atlanticfcu_consolidatedfcu_heartlandcu_midminnesotafcu_sanfranciscofcu_vantagewestcu'):
    return_dict = {}
    print(f'We are loading for parsed files')
    for table in ['inquiry', 'trade', 'collection', 'bankruptcy']:
        print(f'Loading for {table}')
        prefix = f'CLIENT/PARSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE={table}/{PULL_NAME}/{archive_date}'
        file_list = s3.list(prefix, recursive = False)
        return_dict[table] = {'file_list': file_list, 'prefix': prefix}
        print(f'Num of files is {len(file_list)}')
    return return_dict
def grab_processed_data_atlantic(s3, archive_date = archive_date):
    return_dict = {}
    print('we are loading processed files')
    for table in ['inquiry', 'trade', 'collection', 'bankruptcy', 'target']:
        print(f'Loading for {table}')
        prefix = f'PROCESSED/DATA/TABLE={table}/VERSION=v2/CLIENT=atlanticfcu/PRODUCT=autoloanv2/BUREAU=equifax/FORMAT=cms_6/PULL_DATE=2026-01-18/PULL_NAME=20260118_atlanticfcu_consolidatedfcu_heartlandcu_midminnesotafcu_sanfranciscofcu_vantagewestcu/ME_VERSION=v2.0.1/{archive_date}/'
        file_list = s3.list(prefix, recursive = False)
        print(f'Num of files is {len(file_list)}')
        return_dict[table] = s3._read_parquet_dataset(prefix)
    return return_dict
dict_of_parsed_files = grab_parsed_data(s3)
dict_of_processed_data = grab_processed_data_atlantic(s3)

## grab all the IDs for atlatnic 


/home/jag/.conda/envs/newest_model_engine/lib/python3.10/site-packages/zaml/common/utils/io.py:17: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


We are loading for parsed files
Loading for inquiry
Num of files is 29
Loading for trade
Num of files is 23
Loading for collection
Num of files is 1
Loading for bankruptcy
Num of files is 1
we are loading processed files
Loading for inquiry
Num of files is 1
Loading for trade
Num of files is 1
Loading for collection
Num of files is 1
Loading for bankruptcy
Num of files is 1
Loading for target
Num of files is 1


In [6]:
target_df = dict_of_processed_data['target'].reset_index()
zest_key_df = target_df[['ZEST_KEY', 'appDate']].rename(columns = {'appDate': 'date_of_request'})

trade_files =  dict_of_parsed_files['trade']['file_list']
trade_prefix = dict_of_parsed_files['trade']['prefix']

In [7]:
# Compute intensive 
import warnings

# This way loads all at once using s3. Could be compute intensive
#current_trade_file_df = s3._read_parquet_dataset(trade_prefix)
trade_df_list = []
for file in trade_files:
    location = f'{trade_prefix}/{file}'
    current_trade_file_df = pd.read_parquet(f's3://{bucket_name}/{location}')
    trade_df_list.append(zest_key_df.merge(current_trade_file_df, on = ['ZEST_KEY'], how = 'inner'))
current_trade_file_df = pd.concat(trade_df_list)

# Prepare the Tradeline Data

In [22]:
trade_df_first_part_of_processing = apply_mapping_json_and_filter(
    df=current_trade_file_df,
    mapping_json_path="../model-engine/model_engine/assets/equifax/cms_6/fe2/trade.json",
    exclude_raw_features={"DATE_OF_REQUEST"},
    exclude_new_features={"date_of_request"},
    extra_keep={"DATE_OF_REQUEST", "DMD_REPORTED", 'date_of_request'},
    verbose=True,
)



[apply_mapping_json_and_filter] Step 1/29
[apply_converter_spec] Using StringConverterV2 | raw_feature='ZEST_KEY' -> new_feature='ZEST_KEY'

[apply_mapping_json_and_filter] Step 2/29
[apply_converter_spec] Using NumericConverterV2 | raw_feature='BALANCE' -> new_feature='balance_amt'

[apply_mapping_json_and_filter] Step 3/29
[apply_converter_spec] Using NumericConverterV2 | raw_feature='CREDIT_LIMIT' -> new_feature='credit_limit'

[apply_mapping_json_and_filter] Step 4/29
[apply_converter_spec] Using NumericConverterV2 | raw_feature='HIGH_CREDIT' -> new_feature='high_credit_amt'

[apply_mapping_json_and_filter] Step 5/29
[apply_converter_spec] Using NumericConverterV2 | raw_feature='PAST_DUE_AMOUNT' -> new_feature='pastDueAmt'

[apply_mapping_json_and_filter] Step 6/29
[apply_converter_spec] Using NumericConverterV2 | raw_feature='SCHEDULED_PAYMENT_AMOUNT' -> new_feature='scheduled_payment_amount'

[apply_mapping_json_and_filter] Step 7/29
[apply_converter_spec] Using DateConverterV2 

In [24]:
trade_df_first_part_of_processing

,dqDate,ZEST_KEY,pastDueAmt,scheduled_payment_amount,date_of_request,chargeoff_amt,lstPmtDate,ecoa,NARRATIVE_CODE_2,termFreqMult,...,PAYMENT_HISTORY_37_48,termFreqStr,PAYMENT_HISTORY_25_36,termDur,closedDate,credit_limit,NARRATIVE_CODE_1,PAYMENT_HISTORY_1_24,openDate,DMD_REPORTED
0,NaT,76159_1_087_02,0.00,0.00,2025-04-17,NaN,2025-02-01,I,NaN,1.00,...,/111111EE111E,NaN,/111111111111,NaN,NaT,1500.00,NaN,111111111111/111111111111,2017-06-12,None
1,NaT,76159_1_087_02,0.00,0.00,2025-04-17,NaN,2023-02-01,I,NaN,1.00,...,/11111111111*,NaN,/111111111111,48.00,2023-02-01,NaN,EP,************/***********1,2020-10-14,None
2,NaT,76159_1_087_02,0.00,0.00,2025-04-17,NaN,2017-01-01,J,NaN,1.00,...,/************,NaN,/************,36.00,2017-03-01,NaN,NaN,************/************,2015-07-18,None
3,NaT,76159_1_087_02,0.00,0.00,2025-04-17,NaN,NaT,I,NaN,1.00,...,/************,NaN,/************,NaN,2007-12-01,0.00,IR,************/************,2007-11-23,None
4,NaT,76392_1_087_02,0.00,0.00,2025-05-01,NaN,2025-02-01,I,NaN,1.00,...,/1111111EE111,NaN,/11111111111E,NaN,NaT,1280.00,NaN,111EEEEEEEEE/E1111E1111E1,2013-09-01,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,NaT,77308_1_087_02,0.00,25.00,2025-06-23,NaN,2025-03-01,I,NaN,1.00,...,/************,M,/************,NaN,NaT,500.00,NaN,1111111EE***/************,2024-05-25,None
239,NaT,77308_1_087_02,0.00,44.00,2025-06-23,NaN,2025-03-01,I,NaN,1.00,...,/11111111****,M,/111111111111,NaN,NaT,1600.00,NaN,111111111111/111111111111,2021-06-02,None
240,2019-09-01,77308_1_087_02,0.00,30.00,2025-06-23,NaN,2025-02-01,I,NaN,1.00,...,/111111111111,M,/111111111111,NaN,NaT,550.00,NaN,111111111111/111111111111,2019-04-21,None
241,NaT,77308_1_087_02,0.00,0.00,2025-06-23,NaN,2025-02-01,A,NaN,1.00,...,/111111111111,NaN,/111111111111,NaN,NaT,6300.00,NaN,111111111111/111111111111,2010-01-24,None


In [23]:
preprocessed_df = apply_preprocess_json_and_filter(
    mapped_df=trade_df_first_part_of_processing,
    mapping_json_path="../model-engine/model_engine/assets/equifax/cms_6/fe2/trade.json",
    keep_original_columns=True,
    extra_keep={"DATE_OF_REQUEST", "rptDate", "DMD_REPORTED", "date_of_request", 'closedDate', 'majordqDate', 'zest_payment_pattern',   "PAYMENT_HISTORY_1_24",
                        "PAYMENT_HISTORY_25_36",
                        "PAYMENT_HISTORY_37_48"},
    strict_keep=False,
    verbose=True,
)


[apply_preprocess_json_and_filter] Step 1/100 | type=DateDiffV2
[apply_preprocess_spec] DateDiffV2 params=['feature', 'reference_feature', 'new_feature']

[apply_preprocess_json_and_filter] Step 2/100 | type=DateDiffV2
[apply_preprocess_spec] DateDiffV2 params=['feature', 'reference_feature', 'new_feature']

[apply_preprocess_json_and_filter] Step 3/100 | type=DateDiffV2
[apply_preprocess_spec] DateDiffV2 params=['feature', 'reference_feature', 'new_feature']

[apply_preprocess_json_and_filter] Step 4/100 | type=PaymentPatternsAggregatorV2
[apply_preprocess_spec] PaymentPatternsAggregatorV2 params=['report_date', 'payment_patterns']

[apply_preprocess_json_and_filter] Step 5/100 | type=CoalesceV2
[apply_preprocess_spec] CoalesceV2 params=['feature_1', 'feature_2', 'new_feature', 'mode', 'input_dtype']

[apply_preprocess_json_and_filter] Step 6/100 | type=DynamicPlaceholderV2
[apply_preprocess_spec] DynamicPlaceholderV2 params=['raw_feature', 'new_value', 'filter_params', 'raw_feature_

# We use the most recent data pull for the target!!

In [149]:
target_df['app_month'] = target_df['appDate'].dt.to_period('M')

# Get all unique months (sorted)
all_months = (
    target_df['app_month']
    .dropna()
    .sort_values()
    .unique()
)

In [150]:
# We can see that we have 3 months of app dates per target

# bascially archive date 2024-12-31 means their application date was between 

In [151]:
all_months

<PeriodArray>
['2025-04', '2025-05', '2025-06']
Length: 3, dtype: period[M]

In [152]:
target_df['final_DQ30_m9_status'].value_counts()

final_DQ30_m9_status
unseason    103
nohit        79
season        4
Name: count, dtype: int64

In [141]:
target_df[['app_month', 'final_DQ30_m9_status']].value_counts()

app_month  final_DQ30_m9_status
2025-03    unseason                42
2025-01    unseason                34
           nohit                   33
2025-02    unseason                27
2025-03    nohit                   27
2025-02    nohit                   20
2025-01    season                  15
2025-02    season                  15
2025-03    season                   5
Name: count, dtype: int64

In [118]:
target_df['appDate'].dtype

dtype('<M8[ns]')

In [117]:
target_df[target_df['final_DQ30_m3_status']=='unseason']

,appId,appDate,flgFunded,flgApproved,final_DQ30_m3,final_DQ30_m3_source,final_DQ30_m3_status,final_DQ30_m3_sample_weight,final_DateDQ30_m3,final_DQ30_m6,...,final_DateCO_m30,final_CO_m36,final_CO_m36_source,final_CO_m36_status,final_CO_m36_sample_weight,final_DateCO_m36,proxy_MSO_target_CO,proxy_CO,final_MSO_target_CO,final_CO
ZEST_KEY,,,,,,,,,,,,,,,,,,,,,
76745_1_087_02,76745,2025-05-22,0,0,NaN,missing,unseason,1,NaT,NaN,...,NaT,NaN,missing,unseason,1,NaT,NaN,0.0,NaN,0.0
76872_1_087_02,76872,2025-05-30,1,1,NaN,missing,unseason,1,NaT,NaN,...,NaT,NaN,missing,unseason,1,NaT,NaN,0.0,NaN,0.0
77113_1_087_02,77113,2025-06-11,0,0,NaN,missing,unseason,1,NaT,NaN,...,NaT,NaN,missing,unseason,1,NaT,NaN,0.0,NaN,0.0
77208_1_087_02,77208,2025-06-16,1,1,NaN,missing,unseason,1,NaT,NaN,...,NaT,NaN,missing,unseason,1,NaT,NaN,0.0,NaN,0.0
77208_2_087_02,77208,2025-06-16,1,1,NaN,missing,unseason,1,NaT,NaN,...,NaT,NaN,missing,unseason,1,NaT,NaN,0.0,NaN,0.0
77229_1_087_02,77229,2025-06-17,1,1,NaN,missing,unseason,1,NaT,NaN,...,NaT,NaN,missing,unseason,1,NaT,NaN,0.0,NaN,0.0
77229_2_087_02,77229,2025-06-17,1,1,NaN,missing,unseason,1,NaT,NaN,...,NaT,NaN,missing,unseason,1,NaT,NaN,0.0,NaN,0.0
77243_2_087_02,77243,2025-06-18,1,1,NaN,missing,unseason,1,NaT,NaN,...,NaT,NaN,missing,unseason,1,NaT,NaN,0.0,NaN,0.0
77243_1_087_02,77243,2025-06-18,1,1,NaN,missing,unseason,1,NaT,NaN,...,NaT,NaN,missing,unseason,1,NaT,NaN,0.0,NaN,0.0
